<font color='red'> Penalty kick application based on 'Professionals Play Minimax' by Ignacio Palacios-Huerta (2003). </font>

### Make Necessary Imports

In [1]:
from bokeh.io import show, output_notebook, save, output_file
import penalty_kick_automated_game
from main_game_figure import Game_fig_configs
from game_stats_figure_1 import Stats_fig_1_configs
from game_stats_figure_2 import Stats_fig_2_configs
from game_stats_figure_3 import Stats_fig_3_configs

### Configure Bokeh For Use In Jupyter Notebook

In [2]:
output_notebook()

Loading BokehJS ...

### Make Game

In [3]:
#Change the necessary keyword arguments in this to test changes to the game fig.
game_fig_configs = Game_fig_configs()

#Change the necessary keyword arguments in this to test changes to the stats fig 1.
fig_1_configs = Stats_fig_1_configs()

#Change the necessary keyword arguments in this to test changes to the stats fig 2.
fig_2_configs = Stats_fig_2_configs()

#Change the necessary keyword arguments in this to test changes to the stats fig 3.
fig_3_configs = Stats_fig_3_configs()

#Change the necessary keyword arguments in this to test changes scr_text and labels.
scr_text_and_labels_configs = penalty_kick_automated_game.Scr_text_and_labels_configs()

#Change the necessary keyword arguments in this to test changes to divs.
gamestate_divs_configs = penalty_kick_automated_game.Gamestate_divs_configs()

#Change the necessary keyword arguments in this to test changes to buttons.
buttons_configs = penalty_kick_automated_game.Button_configs()

#Change the necessary keyword arguments in this to test changes to sliders.
sliders_configs = penalty_kick_automated_game.Slider_configs()

#Change the necessary keyword arguments in this to test changes to the CPU strategy dropdown menu.
dropdown_configs = penalty_kick_automated_game.Strategy_dropdown_configs()

#Change the necessary keyword arguments in this to test changes to the automation distribution table.
distribution_table_configs = penalty_kick_automated_game.Distribution_table_configs()

#Change the necessary keyword arguments in this to test changes to the automation table.
automation_table_configs = penalty_kick_automated_game.Automation_table_configs()

#Change the necessary keyword arguments in this to test changes to the layout of the game.
layout_configs = penalty_kick_automated_game.Layout_configs()

grid1 = penalty_kick_automated_game.make_game(game_figure_configs = game_fig_configs,
                                              stats_figure_1_configs = fig_1_configs,
                                              stats_figure_2_configs = fig_2_configs,
                                              stats_figure_3_configs = fig_3_configs,
                                              scrtxt_labels_configs = scr_text_and_labels_configs,
                                              divs_configs = gamestate_divs_configs,
                                              button_configs = buttons_configs,
                                              slider_configs = sliders_configs,
                                              strategy_dropdown_configs = dropdown_configs,
                                              distribution_table_configs = distribution_table_configs,
                                              automation_table_configs = automation_table_configs,
                                              layout_configs = layout_configs)

### Show Game

In [4]:
show(grid1)

### Create HTML File

In [5]:
output_file("penalty_kick_game.html")
save(grid1)

'C:\\Users\\kylek\\Documents\\GitHub\\engri-1101-labs\\game_theory\\penalty_kick_game.html'